In [2]:
import numpy as np
from paramagpy import protein, dataparse, fit, metal
import matplotlib as mt
import copy as c
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit as cf
import scipy.stats as sct
from tqdm import tqdm
import multiprocessing

In [3]:
import warnings
from Bio import PDB

# Suppress PDBConstructionWarning
from Bio.PDB.PDBExceptions import PDBConstructionWarning
warnings.simplefilter('ignore', PDBConstructionWarning)

In [4]:
def extract_pos(info):
    pos = []
    dims = ['x','y','z']
    for i in info.strip().split('\n')[2:5]:
        i = i.split()
        if i[1] in dims:
            pos.append(i[-1])
    if len(pos)==3:
        return np.array(pos).astype(float)
    else:
        raise ValueError('problem with mfit.info')
        
def get_qfactor(data):
    return np.sqrt( np.sum(np.square(data['exp']-data['cal'])) / np.sum(np.square(data['exp']+data['cal'])) )

def extract_pc(info):
    pos = []
    pc = ['ax', 'rh']
    for i in info.strip().split('\n')[:2]:
        i = i.split()
        if i[1] in pc:
            pos.append(i[-1])
    if len(pos)==2:
        return np.array(pos).astype(float)
    else:
        raise ValueError('problem with mfit.info')
        
def get_ranges(nobs, ncpus):
    starts = np.arange(0, nobs, nobs//ncpus)[:-1]
    ends = np.empty((ncpus))
    ends[:-1] = starts[1:]
    ends[-1] = nobs
    return starts.astype(int), ends.astype(int)

<font size=3>
    since, the whole point is to check if aI dependent matching is meaningful, then only one simulation is being used, apo is best for this case <br>
    three cam datasets will be used: HN static (or full), leu-substrate-free, leu-with-cam <br><br>

## with HN datasets (static)

In [4]:
nobs = 195001
ncpus = 16
pcoms = np.zeros((nobs,2))
qscores = np.zeros((nobs,2))
dists = np.zeros((nobs,2))

prefix = 'saved_pcs/1_frames_individual/fr'
suffix = '.pdb'

pcs = dataparse.read_pcs('saved_pcs/0_data_from_ubbink/static_exp_closed_HN.npc')

def worker(start, end, sa1, sa2, sa3, sa4, sa5, sa6):
    for a in range(start,end):
        coords = protein.load_pdb(f'{prefix}{a}{suffix}')
        parsed = coords.parse(pcs)
        mstart = metal.Metal()
        mstart.position = coords[0]['A'][195]['CA'].position
        [guess], [data] = fit.svd_gridsearch_fit_metal_from_pcs([mstart], [parsed], 
                                                                radius=10, points=10, verbose=False)
        [mfit], [data] = fit.nlr_fit_metal_from_pcs([guess], [parsed])
        #
        info = mfit.info()
        pos = extract_pos(info)
        pc = extract_pc(info)
        #
        sa1[a] = pc[0]
        sa2[a] = pc[1]
        sa3[a] = fit.qfactor(data)
        sa4[a] = get_qfactor(data)
        sa5[a] = np.linalg.norm(pos-coords[0]['A'][195]['CA'].position*10**10)
        sa6[a] = np.linalg.norm(pos-coords[0]['A'][199]['CA'].position*10**10)


if __name__ == '__main__':
    starts, ends = get_ranges(nobs, ncpus)
    #
    sa1 = multiprocessing.Array('d', nobs)
    sa2 = multiprocessing.Array('d', nobs)
    sa3 = multiprocessing.Array('d', nobs)
    sa4 = multiprocessing.Array('d', nobs)
    sa5 = multiprocessing.Array('d', nobs)
    sa6 = multiprocessing.Array('d', nobs)
    #
    processes = []
    for job in range(ncpus):
        p = multiprocessing.Process(target=worker, 
                                    args=(starts[job], ends[job], sa1, sa2, sa3, sa4, sa5, sa6),
                                   )
        processes.append(p)
        p.start()
    #
    for p in processes:
        p.join()
    #
    pcoms[:,0] = np.frombuffer(sa1.get_obj())
    pcoms[:,1] = np.frombuffer(sa2.get_obj())
    qscores[:,0] = np.frombuffer(sa3.get_obj())
    qscores[:,1] = np.frombuffer(sa4.get_obj())
    dists[:,0] = np.frombuffer(sa5.get_obj())
    dists[:,1] = np.frombuffer(sa6.get_obj())

In [5]:
np.save('saved_pcs/2_output_files/pcoms_static.npy', pcoms)
np.save('saved_pcs/2_output_files/qscores_static.npy', qscores)
np.save('saved_pcs/2_output_files/dists_static.npy', dists)

## with HN dataset (both)

In [ ]:
nobs = 195001
ncpus = 16
pcoms = np.zeros((nobs,2))
qscores = np.zeros((nobs,2))
dists = np.zeros((nobs,2))

prefix = 'saved_pcs/1_frames_individual/fr'
suffix = '.pdb'

pcs = dataparse.read_pcs('saved_pcs/0_data_from_ubbink/both_exp_closed_HN.npc')

def worker(start, end, sa1, sa2, sa3, sa4, sa5, sa6):
    for a in range(start,end):
        coords = protein.load_pdb(f'{prefix}{a}{suffix}')
        parsed = coords.parse(pcs)
        mstart = metal.Metal()
        mstart.position = coords[0]['A'][195]['CA'].position
        [guess], [data] = fit.svd_gridsearch_fit_metal_from_pcs([mstart], [parsed], 
                                                                radius=10, points=10, verbose=False)
        [mfit], [data] = fit.nlr_fit_metal_from_pcs([guess], [parsed])
        #
        info = mfit.info()
        pos = extract_pos(info)
        pc = extract_pc(info)
        #
        sa1[a] = pc[0]
        sa2[a] = pc[1]
        sa3[a] = fit.qfactor(data)
        sa4[a] = get_qfactor(data)
        sa5[a] = np.linalg.norm(pos-coords[0]['A'][195]['CA'].position*10**10)
        sa6[a] = np.linalg.norm(pos-coords[0]['A'][199]['CA'].position*10**10)


if __name__ == '__main__':
    starts, ends = get_ranges(nobs, ncpus)
    #
    sa1 = multiprocessing.Array('d', nobs)
    sa2 = multiprocessing.Array('d', nobs)
    sa3 = multiprocessing.Array('d', nobs)
    sa4 = multiprocessing.Array('d', nobs)
    sa5 = multiprocessing.Array('d', nobs)
    sa6 = multiprocessing.Array('d', nobs)
    #
    processes = []
    for job in range(ncpus):
        p = multiprocessing.Process(target=worker, 
                                    args=(starts[job], ends[job], sa1, sa2, sa3, sa4, sa5, sa6),
                                   )
        processes.append(p)
        p.start()
    #
    for p in processes:
        p.join()
    #
    pcoms[:,0] = np.frombuffer(sa1.get_obj())
    pcoms[:,1] = np.frombuffer(sa2.get_obj())
    qscores[:,0] = np.frombuffer(sa3.get_obj())
    qscores[:,1] = np.frombuffer(sa4.get_obj())
    dists[:,0] = np.frombuffer(sa5.get_obj())
    dists[:,1] = np.frombuffer(sa6.get_obj())

In [ ]:
np.save('saved_pcs/2_output_files/pcoms_both.npy', pcoms)
np.save('saved_pcs/2_output_files/qscores_both.npy', qscores)
np.save('saved_pcs/2_output_files/dists_both.npy', dists)

## with Leu-N substrate-free

In [8]:
nobs = 195001
ncpus = 16
pcoms = np.zeros((nobs,2))
qscores = np.zeros((nobs,2))
dists = np.zeros((nobs,2))

prefix = 'saved_pcs/1_frames_individual/fr'
suffix = '.pdb'

pcs = dataparse.read_pcs('saved_pcs/0_data_from_ubbink/observed_pcs_substrate-free_errors.npc')

def worker(start, end, sa1, sa2, sa3, sa4, sa5, sa6):
    for a in range(start,end):
        coords = protein.load_pdb(f'{prefix}{a}{suffix}')
        parsed = coords.parse(pcs)
        mstart = metal.Metal()
        mstart.position = coords[0]['A'][195]['CA'].position
        [guess], [data] = fit.svd_gridsearch_fit_metal_from_pcs([mstart], [parsed], 
                                                                radius=10, points=10, verbose=False)
        [mfit], [data] = fit.nlr_fit_metal_from_pcs([guess], [parsed])
        #
        info = mfit.info()
        pos = extract_pos(info)
        pc = extract_pc(info)
        #
        sa1[a] = pc[0]
        sa2[a] = pc[1]
        sa3[a] = fit.qfactor(data)
        sa4[a] = get_qfactor(data)
        sa5[a] = np.linalg.norm(pos-coords[0]['A'][195]['CA'].position*10**10)
        sa6[a] = np.linalg.norm(pos-coords[0]['A'][199]['CA'].position*10**10)


if __name__ == '__main__':
    starts, ends = get_ranges(nobs, ncpus)
    #
    sa1 = multiprocessing.Array('d', nobs)
    sa2 = multiprocessing.Array('d', nobs)
    sa3 = multiprocessing.Array('d', nobs)
    sa4 = multiprocessing.Array('d', nobs)
    sa5 = multiprocessing.Array('d', nobs)
    sa6 = multiprocessing.Array('d', nobs)
    #
    processes = []
    for job in range(ncpus):
        p = multiprocessing.Process(target=worker, 
                                    args=(starts[job], ends[job], sa1, sa2, sa3, sa4, sa5, sa6),
                                   )
        processes.append(p)
        p.start()
    #
    for p in processes:
        p.join()
    #
    pcoms[:,0] = np.frombuffer(sa1.get_obj())
    pcoms[:,1] = np.frombuffer(sa2.get_obj())
    qscores[:,0] = np.frombuffer(sa3.get_obj())
    qscores[:,1] = np.frombuffer(sa4.get_obj())
    dists[:,0] = np.frombuffer(sa5.get_obj())
    dists[:,1] = np.frombuffer(sa6.get_obj())

Line ignored while reading file: saved_pcs/0_data_from_ubbink/observed_pcs_substrate-free_errors.npc
134 N 

Line ignored while reading file: saved_pcs/0_data_from_ubbink/observed_pcs_substrate-free_errors.npc
289 N 



In [9]:
np.save('saved_pcs/2_output_files/pcoms_substrate-free.npy', pcoms)
np.save('saved_pcs/2_output_files/qscores_substrate-free.npy', qscores)
np.save('saved_pcs/2_output_files/dists_substrate-free.npy', dists)

## with Leu-N +cam

In [10]:
nobs = 195001
ncpus = 16
pcoms = np.zeros((nobs,2))
qscores = np.zeros((nobs,2))
dists = np.zeros((nobs,2))

prefix = 'saved_pcs/1_frames_individual/fr'
suffix = '.pdb'

pcs = dataparse.read_pcs('saved_pcs/0_data_from_ubbink/observed_pcs_with_cam_errors.npc')

def worker(start, end, sa1, sa2, sa3, sa4, sa5, sa6):
    for a in range(start,end):
        coords = protein.load_pdb(f'{prefix}{a}{suffix}')
        parsed = coords.parse(pcs)
        mstart = metal.Metal()
        mstart.position = coords[0]['A'][195]['CA'].position
        [guess], [data] = fit.svd_gridsearch_fit_metal_from_pcs([mstart], [parsed], 
                                                                radius=10, points=10, verbose=False)
        [mfit], [data] = fit.nlr_fit_metal_from_pcs([guess], [parsed])
        #
        info = mfit.info()
        pos = extract_pos(info)
        pc = extract_pc(info)
        #
        sa1[a] = pc[0]
        sa2[a] = pc[1]
        sa3[a] = fit.qfactor(data)
        sa4[a] = get_qfactor(data)
        sa5[a] = np.linalg.norm(pos-coords[0]['A'][195]['CA'].position*10**10)
        sa6[a] = np.linalg.norm(pos-coords[0]['A'][199]['CA'].position*10**10)


if __name__ == '__main__':
    starts, ends = get_ranges(nobs, ncpus)
    #
    sa1 = multiprocessing.Array('d', nobs)
    sa2 = multiprocessing.Array('d', nobs)
    sa3 = multiprocessing.Array('d', nobs)
    sa4 = multiprocessing.Array('d', nobs)
    sa5 = multiprocessing.Array('d', nobs)
    sa6 = multiprocessing.Array('d', nobs)
    #
    processes = []
    for job in range(ncpus):
        p = multiprocessing.Process(target=worker, 
                                    args=(starts[job], ends[job], sa1, sa2, sa3, sa4, sa5, sa6),
                                   )
        processes.append(p)
        p.start()
    #
    for p in processes:
        p.join()
    #
    pcoms[:,0] = np.frombuffer(sa1.get_obj())
    pcoms[:,1] = np.frombuffer(sa2.get_obj())
    qscores[:,0] = np.frombuffer(sa3.get_obj())
    qscores[:,1] = np.frombuffer(sa4.get_obj())
    dists[:,0] = np.frombuffer(sa5.get_obj())
    dists[:,1] = np.frombuffer(sa6.get_obj())

In [11]:
np.save('saved_pcs/2_output_files/pcoms_with_cam.npy', pcoms)
np.save('saved_pcs/2_output_files/qscores_with_cam.npy', qscores)
np.save('saved_pcs/2_output_files/dists_with_cam.npy', dists)

## with Leu-N +cam +pdx

In [5]:
nobs = 195001
ncpus = 16
pcoms = np.zeros((nobs,2))
qscores = np.zeros((nobs,2))
dists = np.zeros((nobs,2))

prefix = 'saved_pcs/1_frames_individual/fr'
suffix = '.pdb'

pcs = dataparse.read_pcs('saved_pcs/0_data_from_ubbink/observed_pcs_with_cam_pdx_errors.npc')

def worker(start, end, sa1, sa2, sa3, sa4, sa5, sa6):
    for a in range(start,end):
        coords = protein.load_pdb(f'{prefix}{a}{suffix}')
        parsed = coords.parse(pcs)
        mstart = metal.Metal()
        mstart.position = coords[0]['A'][195]['CA'].position
        [guess], [data] = fit.svd_gridsearch_fit_metal_from_pcs([mstart], [parsed], 
                                                                radius=10, points=10, verbose=False)
        [mfit], [data] = fit.nlr_fit_metal_from_pcs([guess], [parsed])
        #
        info = mfit.info()
        pos = extract_pos(info)
        pc = extract_pc(info)
        #
        sa1[a] = pc[0]
        sa2[a] = pc[1]
        sa3[a] = fit.qfactor(data)
        sa4[a] = get_qfactor(data)
        sa5[a] = np.linalg.norm(pos-coords[0]['A'][195]['CA'].position*10**10)
        sa6[a] = np.linalg.norm(pos-coords[0]['A'][199]['CA'].position*10**10)


if __name__ == '__main__':
    starts, ends = get_ranges(nobs, ncpus)
    #
    sa1 = multiprocessing.Array('d', nobs)
    sa2 = multiprocessing.Array('d', nobs)
    sa3 = multiprocessing.Array('d', nobs)
    sa4 = multiprocessing.Array('d', nobs)
    sa5 = multiprocessing.Array('d', nobs)
    sa6 = multiprocessing.Array('d', nobs)
    #
    processes = []
    for job in range(ncpus):
        p = multiprocessing.Process(target=worker, 
                                    args=(starts[job], ends[job], sa1, sa2, sa3, sa4, sa5, sa6),
                                   )
        processes.append(p)
        p.start()
    #
    for p in processes:
        p.join()
    #
    pcoms[:,0] = np.frombuffer(sa1.get_obj())
    pcoms[:,1] = np.frombuffer(sa2.get_obj())
    qscores[:,0] = np.frombuffer(sa3.get_obj())
    qscores[:,1] = np.frombuffer(sa4.get_obj())
    dists[:,0] = np.frombuffer(sa5.get_obj())
    dists[:,1] = np.frombuffer(sa6.get_obj())

In [6]:
np.save('saved_pcs/2_output_files/pcoms_with_pdx.npy', pcoms)
np.save('saved_pcs/2_output_files/qscores_with_pdx.npy', qscores)
np.save('saved_pcs/2_output_files/dists_with_pdx.npy', dists)